In [ ]:
from src.Archives.moodle_archive import MoodleArchive
from src.Archives.question_bank import *
import src.Archives.graph_structure as gs

import pandas as pd

In [ ]:
courseID = 1
#role = 'Teacher'
role = 'Student'

In [ ]:
ARCHIVE_FILENAME = 'sauvegarde-moodle2-course-1-nu.mbz'
ARCHIVE_PATH = os.path.join("../Datasets", "MoodleCourseBackups")
ARCHIVE_DESTINATION_PATH = os.path.join("../Datasets", "MoodleCourseArchives")
archive = MoodleArchive(ARCHIVE_PATH, ARCHIVE_FILENAME, ARCHIVE_DESTINATION_PATH)

In [ ]:
# course chaptering
gs.course_chaptering(archive)

In [ ]:
course_structure = gs.create_course_structure(archive)
figure = gs.plot_graph(course_structure, figsize=(180, 30), textlen=35)

In [ ]:
PATH = '../Datasets/xAPI_logs/PREPROCESSED_files'
df = pd.read_pickle(f"{PATH}/cours_{courseID}.pkl")

In [ ]:
df = df.loc[(df.CourseID == courseID) & ((df.Role == 'Student')
                                         | (df.Role == 'Editing Teacher')
                                         | (df.Role == 'Teacher'))]

df = df[['User', 'Timestamp', 'Context', 'Description', 'Component',
         'Event_name', 'ObjectID', 'ItemID', 'QuestionID', 'Status', 'Role', 'Group']]


In [ ]:
# select the components to check
excluded_components = ['Role', 'Groups', 'Grades', 'User profile', 'Enrolment', 'Attendance']

components = list(set(df.Component.unique()) - set(excluded_components))

# filter selected components, available activities, and students.
df = df.loc[(df.Component.isin(components)) & (df.Status != 'DELETED') & (df.Role.str.contains(role))].copy()


In [ ]:
for node in course_structure.nodes:
    settings = course_structure.nodes[node]['settings']
    component = node.split('_')[0].capitalize()
    if component == 'Assign':
        component = 'Assignment'
    if component == 'Lti':
        component = 'External tool'
    if component == 'Resource':
        component = 'File'
    if component == 'Url':
        component = 'URL'
    if component == 'Scorm':
        component = 'SCORM package'
    if component == 'Data':
        component = 'Database'
    if component == 'Choicegroup':
        component = 'Group choice'
    if component in components and node != 'course':
        object_id = int(node.split('_')[1])
        condition = (df.Component == component) & (df.ObjectID == object_id)
        df.loc[
            condition, 'Context'] = settings.name  # TODO: remove in the final code, but here is needed to fix hashing "context" errors.
        df.loc[condition, 'Visibility'] = settings.visible
        df.loc[condition, 'Availability'] = settings.availability
        df.loc[condition, 'Section_number'] = settings.sectionnumber

df.loc[(df.Component == 'Course'), 'Visibility'] = True
df.loc[(df.Component == 'Course'), 'Section_number'] = -1

for section in archive.section_list:
    number = section.settings.number
    name = section.settings.name
    df.loc[df.Section_number == number, 'Section_name'] = name

df.loc[df.Section_number == -1, 'Section_name'] = 'Home page'


In [ ]:
PATH_GRAPH = '../Datasets/Graph_Integrated_Logs'
df.to_pickle(f"{PATH_GRAPH}/course_{courseID}_{role}_graph_integrated.pkl")
